<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Github - Get profiles from teams
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Github/Github_Create_issue.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

This notebook enables you to get a dataframe of all the team members in a Github organization.

**Tags:** #github #team

## Input

### Import libraries

In [ ]:
import pandas as pd
import requests
import naas

### Setup Github

**How to find your personal access token on Github?** 
- First we need to create a personal access token to get the details of our organization from here: https://github.com/settings/tokens
- You will be asked to select scopes for the token. Which scopes you choose will determine what information and actions you will be able to perform against the API. 
- You should be careful with the ones prefixed with write:, delete: and admin: as these might be quite destructive. 
- You can find description of each scope in docs here (https://docs.github.com/en/developers/apps/building-oauth-apps/scopes-for-oauth-apps).


In [ ]:
TEAM_URL = "https://api.github.com/orgs/jupyter-naas/teams"
github_token = naas.secret.get("github_token")

## Model

In [ ]:
query_url = TEAM_URL
params = {
    "state": "open",
}
headers = {'Authorization': f'token {github_token}'}

In [ ]:
main_data = requests.get(query_url, headers=headers, params=params).json()
members_details, GITHUB = [],'jupyter-naas'
for team_info in main_data:
    members_details.append((team_info['name'], team_info['slug'], team_info['description'], team_info['members_url'].strip("{/member}")))

In [ ]:
def retrieve_members(team, slug, description, page_link, page_number=1):
    try:
        members = requests.get(page_link, headers=headers, params=params).json()
    except:
        return teams, slugs, descriptions, member_profiles
    
    for member in members:
        member_profiles.append(member['url'])
        teams.append(team)
        slugs.append(slug)
        descriptions.append(description)
    
    if len(members)==30:  ## default members per page shown are 30 ##
        page_number+=1
        if page_number==2:
            page_link+= f'?page={page_number}'
        else:
            index = page_link.find('?page=')
            page_link = page_link[:index+6] + str(page_number)
        return retrieve_members(team, slug, description, page_link, page_number)
    else:
        return teams, slugs, descriptions, member_profiles

member_profiles, teams, slugs, descriptions, page_number=[],[],[],[],1

for info in members_details:
    teams, slugs, descriptions, member_profiles = retrieve_members(info[0], info[1], info[2], info[3], page_number)

    
data = pd.DataFrame(columns=['TEAM', 'SLUG','TEAM_DESCRIPTION', 'member_profile'])
data['TEAM'], data['SLUG'], data['TEAM_DESCRIPTION'], data['member_profile'] = teams, slugs, descriptions, member_profiles
data['GITHUB'] = 'jupyter-naas'

print(data.shape)

In [ ]:
for idx, profile in enumerate(data['member_profile']):
    details = requests.get(profile, headers=headers, params= params).json()
    data.loc[idx,'NAME'], data.loc[idx,'EMAIL'], data.loc[idx,'LOCATION'] = details['name'], details['email'], details['location']
    data.loc[idx,'ORGANIZATION'], data.loc[idx,'BIO'], data.loc[idx,'LOGIN_NAME'] = details['company'], details['bio'], details['login']
    data.loc[idx,'TWITTER'], data.loc[idx,'CREATED_AT'], data.loc[idx,'UPDATED_AT'] = details['twitter_username'], details['created_at'], details['updated_at']

## Output

In [ ]:
print(f'Dataset size -> {data.shape}')
data